In [1]:
import pandas as pd
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import replicate_row,base_selection,expand_rows,clean_data
from streamlit_app.teacher_alocation import TeacherScheduler

In [4]:
professores_raw = pd.read_excel('/home/cayena/Downloads/professores_final.xlsx')
professores_raw.columns = professores_raw.columns.astype(str)

aulas_raw = pd.read_excel('/home/cayena/Downloads/rotas_finais.xlsx')

In [5]:
aulas_simples,doub,tri=base_selection(aulas_raw)

# transformando aulas duplas/triplas em 2/3 linhas
aulas_duplicadas = expand_rows(doub, lambda row: replicate_row(row, times=2))
aulas_triplicadas = expand_rows(tri, lambda row: replicate_row(row, times=3))

# juntando todas as linhas
df_tratado = pd.concat([aulas_simples, aulas_duplicadas, aulas_triplicadas], ignore_index=True)

df_final = clean_data(df_tratado)

df_final.head()

,nome grupo,horario,unidade,dias da semana,modalidade,grupo,n aulas,teacher,status,validacao,intenviso,horario_tratado
0,ANDRESSA VIP EAD - 1X,11:00:00,Vicentina,SEGUNDA,Inglês,VIP,1,NaN,ONLINE,-,0,1900-01-01 11:00:00
2,KIKO VIP P - 1X,08:00:00,Vicentina,QUINTA,Inglês,VIP - In Company,1,NaN,PRESENCIAL,BRUNO MORGILLO,0,1900-01-01 08:00:00
4,VIVIANE BENEFICIAR VIP EAD - 2X,13:30:00,Jardim,TERÇA,Inglês,VIP,2,NaN,ONLINE,-,0,1900-01-01 13:30:00
6,VIVIANE BENEFICIAR VIP EAD - 2X,13:30:00,Jardim,QUINTA,Inglês,VIP,2,NaN,ONLINE,-,0,1900-01-01 13:30:00
8,KIDS DAISY P - 2X,09:30:00,Jardim,TERÇA,Kids,Grupo,2,NaN,PRESENCIAL,CENKAY,0,1900-01-01 09:30:00


In [6]:
Ts = TeacherScheduler(df_final, professores_raw)
base_alocada = Ts.schedule_teachers()
base_alocada


Starting CP-SAT solver v9.7.2996
Parameters: max_time_in_seconds: 3600 log_search_progress: true search_branching: FIXED_SEARCH num_search_workers: 8

Initial optimization model '': (model_fingerprint: 0xed9ff9e8a868ed73)
#Variables: 3'961 (#bools: 3'960 in objective)
  - 3'960 Booleans in [0,1]
  - 1 constants in {1} 
#kBoolOr: 1'560 (#literals: 1'560)
#kLinear1: 2'439
#kLinear2: 930
#kLinear3: 1'680
#kLinearN: 2'772 (#terms: 25'410)

Starting presolve at 0.00s
[ExtractEncodingFromLinear] #potential_supersets=3174 #potential_subsets=0 #at_most_one_encodings=0 #exactly_one_encodings=0 #unique_terms=0 #multiple_terms=0 #literals=0 time=0.000161409s
[Symmetry] Graph for symmetry has 7757 nodes and 22539 arcs.
[Symmetry] Symmetry computation done. time: 0.0263089 dtime: 0.0702645
[Symmetry] #generators: 134, average support size: 43.8806
[Symmetry] The model contains 667 duplicate constraints !
[Symmetry] 188 orbits with sizes: 112,58,56,56,48,42,42,42,42,42,...
[Symmetry] Num fixable by

,Professor,nome grupo
0,ANA - MBA,ANDRESSA VIP EAD - 1X
1,ANA - MBA,KIKO VIP P - 1X
2,RYAN,VIVIANE BENEFICIAR VIP EAD - 2X
3,MARIA,KIDS DAISY P - 2X
4,MARIA,DELTA P - 4X
...,...,...
126,CLAUDIO,SAN JUAN EAD - 3X
127,LEONARDO,BARCELONA P - 3X
128,LEONEL,GIBRALTAR P - 3X
129,MADDIE,SCOTT EAD - 3X


In [7]:
final = pd.merge(aulas_raw, base_alocada, on='nome grupo', how='left')

In [8]:
final[final['Professor']=='BRUNO MORGILLO']

,nome grupo,horario,unidade,dias da semana,modalidade,grupo,n aulas,teacher,status,validacao,Professor
34,BRISBANE P - 3X,08:00:00,Satélite,Saturday - Triple,Inglês,Grupo,3,NaN,PRESENCIAL,DARLENE,BRUNO MORGILLO
45,EDINBURGH P - 3X,19:00:00,Jardim,2ª ● 4ª ● 5ª,Inglês,Grupo,3,NaN,PRESENCIAL,CENKAY,BRUNO MORGILLO
58,CURAÇAU EAD - 4X,08:00:00,Vicentina,2ª ● 3ª ● 4ª ● 5ª,Inglês,Grupo,4,NaN,ONLINE,FERNANDO,BRUNO MORGILLO
74,SEATTLE P - 3X,15:00:00,Vicentina,2ª ● 4ª ● 5ª,Inglês,Grupo,3,NaN,PRESENCIAL,LEONEL,BRUNO MORGILLO
76,FELIPE MASCAGNA VIP P - 2X,16:00:00,Vicentina,2ª ● 4ª,Inglês,VIP,2,NaN,PRESENCIAL,LEONEL,BRUNO MORGILLO
78,LANCASTER P - 4X,17:00:00,Vicentina,2ª ● 3ª ● 4ª ● 5ª,Inglês,Grupo,4,NaN,PRESENCIAL,TELMA,BRUNO MORGILLO
94,IRELAND EAD - 4X,18:00:00,Vicentina,2ª ● 3ª ● 4ª ● 5ª,Inglês,Grupo,4,NaN,ONLINE,MADDIE,BRUNO MORGILLO
99,GEISSE VIP CONV EAD - 1X,09:00:00,Vicentina,4ª,CONV - Ing Intermed,VIP,1,NaN,ONLINE,MARIA,BRUNO MORGILLO
104,WISCONSIN EAD - 3X,20:00:00,Jardim,2ª ● 4ª ● 5ª,Inglês,Grupo,3,NaN,ONLINE,LUCIANO,BRUNO MORGILLO


In [9]:
final[final['Professor'].isnull()]

,nome grupo,horario,unidade,dias da semana,modalidade,grupo,n aulas,teacher,status,validacao,Professor
122,NEBRASKA P - 3X,08:00:00,Satélite,Saturday - Triple,Inglês,Grupo,3,NaN,PRESENCIAL,TELMA,NaN


In [10]:
professores_raw[professores_raw['TEACHER']=='BRUNO MORGILLO'][['10:00:00','QUARTA','SÁBADO']]

,10:00:00,QUARTA,SÁBADO
5,1,1,1.0


In [11]:
aulas_raw[aulas_raw['nome grupo']=='CHARLESTON P - 3X']

,nome grupo,horario,unidade,dias da semana,modalidade,grupo,n aulas,teacher,status,validacao
67,CHARLESTON P - 3X,08:00:00,Satélite,Saturday - Triple,Inglês,Grupo,3,NaN,PRESENCIAL,LEONARDO


In [12]:
restricoes = []

# Iterar sobre professores e dias da semana
for i in professores_raw['TEACHER'].unique():
    for x in df_final['dias da semana'].unique():
        # Obter as turmas do dia específico
        turmas_do_dia = df_final[df_final['dias da semana'] == x]['nome grupo'].unique()
        
        # Verificar se o professor está indisponível no dia
        if professores_raw[professores_raw['TEACHER'] == i][x].values[0] == 0:
            for g in turmas_do_dia:
                # Adicionar restrição à lista
                restricoes.append({'Professor': i, 'nome grupo': g, 'Dia': x})

# Criar DataFrame a partir das restrições
df_restricoes = pd.DataFrame(restricoes)

In [13]:
base_alocada

,Professor,nome grupo
0,ANA - MBA,ANDRESSA VIP EAD - 1X
1,ANA - MBA,KIKO VIP P - 1X
2,RYAN,VIVIANE BENEFICIAR VIP EAD - 2X
3,MARIA,KIDS DAISY P - 2X
4,MARIA,DELTA P - 4X
...,...,...
126,CLAUDIO,SAN JUAN EAD - 3X
127,LEONARDO,BARCELONA P - 3X
128,LEONEL,GIBRALTAR P - 3X
129,MADDIE,SCOTT EAD - 3X
